In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import minimize
from scipy.optimize import differential_evolution

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')


## modelagem GLM

In [2]:
%%time
df_mod_mp_feb23jul24_v2 = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_mp_feb23jul24_v2.csv",
                     encoding='latin-1',
                     delimiter='|')

df_mod_mp_feb23jul24_v2.shape

CPU times: total: 6.66 s
Wall time: 7.12 s


(575924, 105)

In [3]:
df_mod_mp_feb23jul24_v2['Grupo_veiculo_new']=df_mod_mp_feb23jul24_v2['novo']

#### prepara base de modelagem glm

In [4]:
df_mod_filtered=df_mod_mp_feb23jul24_v2[["premio_ofertado","premio_total", 'premio_creserva','premio_demais_cob',"teste_validacao","ind_renovacao","MD_sexo","cod_corr",
                                "Cod_End","MD_sin_5a_cas","MD_tempo_casa_2","MD_idade_veic","MD_faixa_IScas",
                                "anomes_dt_ter" ,"MD_idade","per_com", "RNS_Casco_calc","MD_menor26","UF","Renovacao_ant",
                                "fx_ir_12m","is_casco_ofer","classe_bonus","MD_fator_ajuste","MD_capac","md_categoria",
                                "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA','MD_est_civil',
                                'peso','Grupo_veiculo_new','vigencia',
                                'Risco_PP_mod', 'Risco_PT_mod','Risco_RF_mod','Risco_DM_mod','Risco_AS_mod',
                                'Risco_DC_mod' ,'Risco_DMO_mod']]

df_mod_filtered['DA_emissao'] =0.2349
df_mod_filtered['DA_producao'] =0.2749
df_mod_filtered['risco_dmais_cob'] =df_mod_filtered['premio_demais_cob']*0.2627*1.4
df_mod_filtered['premio_anterior'] =df_mod_filtered['premio_total']
df_mod_filtered['Act_cost']  = df_mod_filtered['Risco_PP_mod'] + df_mod_filtered['Risco_PT_mod'] + df_mod_filtered['Risco_RF_mod']+ df_mod_filtered['Risco_DM_mod']+ df_mod_filtered['Risco_AS_mod'] + df_mod_filtered['Risco_DC_mod'] +df_mod_filtered['Risco_DMO_mod'] + df_mod_filtered['risco_dmais_cob']
df_mod_filtered['Premio_IC100']  = df_mod_filtered['Act_cost']/(1-df_mod_filtered['per_com']/100-df_mod_filtered['DA_producao'] )
df_mod_filtered["premio"] = df_mod_filtered["premio_ofertado"]

In [5]:
df_mod_filtered = df_mod_filtered[df_mod_filtered['per_com'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['FABRICA'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['MD_est_civil'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['Renovacao_ant'].notna()]
df_mod_filtered = df_mod_filtered.query('premio_total !=0')
df_mod_filtered = df_mod_filtered.query('vigencia == "Anual"')
df_mod_filtered = df_mod_filtered.query('premio_creserva >= 0')
df_mod_filtered = df_mod_filtered[~df_mod_filtered['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]
df_mod_filtered = df_mod_filtered.query('premio_total >100')
df_mod_filtered = df_mod_filtered.query('premio_ofertado >100')
df_mod_filtered = df_mod_filtered.query('MD_idade !="ERRO"')
df_mod_filtered = df_mod_filtered.query('is_casco_ofer > 0')
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']>0)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']<50)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total <=2.3)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total >=0.7)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.UF != "SP") | (~df_mod_filtered.Grupo_veiculo_new.isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]

df_mod_filtered.shape

(363905, 50)

In [6]:
#df_mod_filtered['ind_renovacao'].describe()

In [7]:
#print(df_mod_filtered['classe_bonus'].value_counts(ascending=False))

In [8]:
df_mod=df_mod_filtered.copy()

In [9]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(int)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01_00_20' if x == '01 - 00-<=020' else
                                                          '02_20_40' if x == '02 - 20-<=040' else
                                                          '03_40_60' if x == '03 - 40-<=060' else
                                                          '04_60_80' if x == '04 - 60-<=080' else
                                                          '05_80_100' if x == '05 - 80-<=100' else
                                                          '02_20_40' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor_Mais' if x == 'Corretor Mais' else 
                                                   'Corretor_Mais' if x == 'CorretorMais'
                                                                   else    'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: 'p_1_10' if x < 10 else
                                                      'p_10_35' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x:     'Sem_Sinistro' if x == 'Sem Sinistro' else
                                                                      '01_05_anos' if x == '01 ano' else
                                                                      '01_05_anos' if x == '02 anos' else
                                                                      '01_05_anos' if x == '03 anos' else
                                                                      '01_05_anos' if x == '04 anos' else
                                                                      '01_05_anos' if x == '05 anos' else '06_inf')


    df['UF_bin'] = df['UF'].apply(lambda x:     'AL_CE_RS'  if x == 'AL' else
                                                'AL_CE_RS'  if x == 'CE' else
                                                'AL_CE_RS'  if x == 'RS' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'AP' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'DF' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'ES' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PA' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PB' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'RN' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'SC' else 
                                                'GO_MA_MG'             if x == 'GO' else 
                                                'GO_MA_MG'             if x == 'MA' else 
                                                'GO_MA_MG'             if x == 'MG' else 
                                                'MT_PR'                if x == 'MT' else
                                                'MT_PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  
                                                'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1_SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2_RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2_RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3_RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3_RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '7 - Renov BB com sinistro' else  
                                                                          '2_RenovPropria')
  
    df['MD_tipo_renov_bin'] = df['MD_tipo_renov'].apply(lambda x:          '2_RenovPropria_sem_sin'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                           '3_RenovPropria_com_sin'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                           'Error')
   
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: 'f_80_101'    if x < 101 else
                                                                   'f_101_120'   if x >= 101  else 
                                                                   'f_80_101')   
    
    return df

In [10]:
df_mod = categorizar(df_mod)

In [11]:
df_train = df_mod.query('teste_validacao == "Trein"')
df_train.shape

(291023, 64)

In [12]:
#print(df_train['fator_ajuste_bin'].value_counts(ascending=False)) 

In [13]:
df_num = df_train[['premio', 'per_com', 'MD_tempo_casa_2', 'is_casco_ofer','Premio_IC100','premio_anterior']]
df_cat = df_train[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_y = df_train[['ind_renovacao']]
df_cat_glm=pd.get_dummies(df_cat)
df_cat_glm=pd.concat([df_cat_y,df_num, df_cat_glm],axis=1)

In [14]:
#df_cat_glm.columns.to_list()

In [15]:
import statsmodels.formula.api as smf
#Modelo GLM de retenção
formula =  'ind_renovacao ~ 1 + premio + per_com + per_com_bin_p_1_10 + canal_bin_Corretor_Mais + Cod_End_bin_13509 + fator_ajuste_bin_f_101_120 + classe_bonus_bin_00  ' 
model_glm2 = smf.logit(formula , data=df_cat_glm).fit()
model_glm2.summary()

Optimization terminated successfully.
         Current function value: 0.638311
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          ind_renovacao   No. Observations:               291023
Model:                          Logit   Df Residuals:                   291015
Method:                           MLE   Df Model:                            7
Date:                Mon, 19 Aug 2024   Pseudo R-squ.:                 0.07598
Time:                        18:49:24   Log-Likelihood:            -1.8576e+05
converged:                       True   LL-Null:                   -2.0104e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.1107      0.015      7.411      0.000       0.081       0.140
premio                        -0.0003   2.65e-06   -113.249      0.000      -0.000      -0.000
per_com                        0.0275      0.001     35.612      0.000       0.026       0.029
per_com_bin_p_1_10             0.5022      0.023     21.924      0.000       0.457       0.547
canal_bin_Corretor_Mais        0.5601      0.009     63.379      0.000       0.543       0.577
Cod_End_bin_13509             -0.0983      0.013     -7.326      0.000      -0.125      -0.072
fator_ajuste_bin_f_101_120     0.3867      0.027     14.362      0.000       0.334       0.439
classe_bonus_bin_00           -0.6541      0.036    -18.098      0.000      -0.725      -0.583
==============================================================================================
"""

## Base Para escorar e otimizar

In [16]:
df_lote_092024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Table 7 Base Case Total  - teste novo.csv",
                     encoding='latin-1',
                     delimiter='|')

In [17]:
df_lote_escorar=df_lote_092024.copy()

In [18]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [19]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [20]:
df_lote = categorizar(df_lote_escorar)

In [21]:
df_cat = df_lote[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_glm=pd.get_dummies(df_cat)
df_lote=pd.concat([df_lote, df_cat_glm],axis=1)

In [22]:
#df_lote_2.columns.to_list()

In [23]:
#df_lote.groupby(["Amostra"])["prob_est" , "Renewal Demand"].describe().T

## Otimização Teste 

In [24]:
#print(df_lote['canal'].value_counts(ascending=False))

In [22]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra == "Otimização Atu"')
df_lote_otim_atu.shape

(10925, 169)

In [26]:
#df_lote_otim_atu.groupby(["Amostra"])["prob_est" , "Demanda_base"].describe().T

In [27]:
df_lote_otim = df_lote_otim_atu.sample(n =10) 
df_lote_otim.shape

(10, 169)

In [ ]:
P=np.array(df_lote_otim['premio'])
X1=np.array(df_lote_otim['per_com'])
X2=np.array(df_lote_otim['per_com_bin_p_1_10'])
X3=np.array(df_lote_otim['canal_bin_Corretor_Mais'])
X4=np.array(df_lote_otim['Cod_End_bin_13509'])
X5=np.array(df_lote_otim['fator_ajuste_bin_f_101_120'])
X6=np.array(df_lote_otim['classe_bonus_bin_00'])

In [23]:
# Define a função de demanda
def demanda_base(X1,X2,X3,X4,X5,X6,P):
    D = np.exp(0.1107 + 0.0275*X1 + 0.5022*X2 + 0.5601*X3 -0.0983*X4 + 0.3867*X5 -0.6541*X6 -0.0003*P  ) / (1 + np.exp(np.exp(0.1107 + 0.0275*X1+0.5022*X2+0.5601*X3-0.0983*X4+0.3867*X5-0.6541*X6 -0.0003*P  ) ))
    return D / (1 + D)
df_lote_otim['Demanda_base']=demanda_base(X1,X2,X3,X4,X5,X6,P)
df_lote_otim['Demanda_base'].sum()

NameError: name 'X1' is not defined

In [ ]:
df_lote_otim['Demanda_base'].mean()

#########################################

In [ ]:
df_relatorio_bc = df_lote_otim.copy()

df_relatorio_bc["per_com_pct"] = df_relatorio_bc['per_com']/100
df_relatorio_bc["premio_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["premio"]
df_relatorio_bc["Act_cost_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["Act_cost"]                                                                
df_relatorio_bc["comissao_valor"] = df_relatorio_bc["per_com_pct"] *df_relatorio_bc["premio"]
df_relatorio_bc["comissao_valor_prob"] = df_relatorio_bc["per_com_pct"] *df_relatorio_bc["premio_prob"]

apolice_ofertada  = np.round(df_relatorio_bc.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_bc["Demanda_base"]),0)
premio_ofertado   = np.mean(df_relatorio_bc["premio"])
retencao_estimada = np.mean(df_relatorio_bc["Demanda_base"])
premio_esperado   = np.sum(df_relatorio_bc["premio_prob"])/np.sum(df_relatorio_bc["Demanda_base"])
risco_ofertado    = np.mean(df_relatorio_bc["Act_cost"])
risco_esperado    = np.sum(df_relatorio_bc["Act_cost_prob"])/np.sum(df_relatorio_bc["Demanda_base"])
sp_ofertado       = np.sum(df_relatorio_bc["Act_cost"])/np.sum(df_relatorio_bc["premio"])
sp_esperado       = np.sum(df_relatorio_bc["Act_cost_prob"])/np.sum(df_relatorio_bc["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_bc["comissao_valor"])/np.sum(df_relatorio_bc["premio"])
comis_esperado    = np.sum(df_relatorio_bc["comissao_valor_prob"])/np.sum(df_relatorio_bc["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

In [ ]:
np.mean(df_relatorio_bc['Renewal Demand'])

In [ ]:
np.mean(df_relatorio_bc['prob_est'])

In [ ]:
np.mean(df_relatorio_bc['Demanda_base'])

In [ ]:
#df2 = df[['premio','is_casco_ofer','Cod_End_bin_Demais','Act_cost']]

In [ ]:
#df2.to_excel(r'analise_teste.xlsx',  header='true')

In [ ]:
#[x for x in df.columns if "com_bin" in x]

In [32]:
df_lote_otim[['premio',
              'per_com',
              'per_com_bin_p_1_10',
              'canal_bin_Corretor_Mais',
              'Cod_End_bin_13509',
              'fator_ajuste_bin_f_101_120',
              'classe_bonus_bin_00',
              'Act_cost']]

,premio,per_com,per_com_bin_p_1_10,canal_bin_Corretor_Mais,Cod_End_bin_13509,fator_ajuste_bin_f_101_120,classe_bonus_bin_00,Act_cost
23347,4216.590000,20.0,0,0,0,0,0,2735.246678
19033,3085.934749,19.2,0,1,1,0,0,1645.111815
1825,1086.120645,10.0,0,0,0,0,0,678.934015
5670,1512.088311,20.0,0,0,0,0,0,793.997572
23926,3131.613839,10.0,0,0,0,0,0,1957.571811
11583,2390.732896,10.0,0,0,1,0,0,1494.447133
7434,1789.918629,10.0,0,0,0,0,0,1118.878135
19744,3970.197000,18.0,0,0,0,0,0,2816.304722
11299,2085.685676,16.0,0,0,1,0,0,1178.620975
26756,4683.245000,20.0,0,0,0,0,1,4652.113609


In [ ]:
sol = Parallel(n_jobs=2)(delayed(fsolve)(equations, x0=p) for p in x0)
print(sol)

In [33]:
# Definir a função de demanda
def demanda_teste(X1, X2, X3, X4, X5, X6, P):
    D = np.exp(0.1107 + 0.0275 * X1 + 0.5022 * X2 + 0.5601 * X3 - 0.0983 * X4 + 0.3867 * X5 - 0.6541 * X6 - 0.0003 * P)
    return D / (1 + D)

In [66]:
df_lote_otim = df_lote_otim_atu.sample(n =200) 

In [67]:
d=demanda_teste(df_lote_otim['per_com'],df_lote_otim['per_com_bin_p_1_10'],df_lote_otim['canal_bin_Corretor_Mais'],df_lote_otim['Cod_End_bin_13509'],df_lote_otim['fator_ajuste_bin_f_101_120'],df_lote_otim['classe_bonus_bin_00'],df_lote_otim['premio'])
print(d.sum()),
print(d.mean())

94.2665999942133
0.4713329999710665


In [68]:
%%time
######
import numpy as np
import sympy as sp
from scipy.optimize import fsolve
import scipy.optimize as optimize

df = df_lote_otim

lista_variaveis_na_ordem =[
              'per_com',
              'per_com_bin_p_1_10',
              'canal_bin_Corretor_Mais',
              'Cod_End_bin_13509',
              'fator_ajuste_bin_f_101_120',
              'classe_bonus_bin_00']

indicadora=np.array(df.loc[:,lista_variaveis_na_ordem])                                             
precos_iniciais=np.array(df['premio'])
tarifa=np.array(df['premio'])
target =0.42*df.shape[0]
custo = df['Act_cost'].mean()



# Definir a função de demanda
def demanda(X1, X2, X3, X4, X5, X6, P):
    D = np.exp(0.1107 + 0.0275 * X1 + 0.5022 * X2 + 0.5601 * X3 - 0.0983 * X4 + 0.3867 * X5 - 0.6541 * X6 - 0.0003 * P)
    return D / (1 + D)

df['Demanda_base']=demanda(df['per_com'],df['per_com_bin_p_1_10'],df['canal_bin_Corretor_Mais'],df['Cod_End_bin_13509'],df['fator_ajuste_bin_f_101_120'],df['classe_bonus_bin_00'],df['premio'])



# Função objetivo (com restrições)
def func(X, indicadora, precos_iniciais):
    P = X[:-1]  
    L = X[-1]
    n = len(P)

    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(indicadora[i, 0], indicadora[i, 1], indicadora[i, 2], indicadora[i, 3], indicadora[i, 4], indicadora[i, 5], P[i])
        objetivo += D_i * (P[i] - custo)
        soma_demandas += D_i


    return objetivo - L * (soma_demandas - target)

# Diferenciação numérica
def dfunc(X, indicadora, precos_iniciais):
    dLambda = np.zeros(len(X))
    h = 1e-3
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X + dX, indicadora, precos_iniciais) - func(X - dX, indicadora, precos_iniciais)) / (2 * h)
    return dLambda

# Otimizar preços
def otimizar_precos(indicadora, precos_iniciais):
    X_inicial = np.append(precos_iniciais, 0)  # Valor inicial para o multiplicador de Lagrange
    
    # Otimização utilizando fsolve
    X_otimizado = optimize.fsolve(dfunc, X_inicial, args=(indicadora, precos_iniciais), xtol=1e-03, maxfev=100)
    
       
       # Retornando o vetor de preços otimizados e o multiplicador de Lagrange
    precos_otimizados = X_otimizado[:-1]
   
    #precos_otimizados = np.maximum(precos_otimizados, 0.9 * tarifa*1.1) 
    
    lambida = X_otimizado[-1]
    
    return precos_otimizados, lambida

# Executando a otimização
precos_otimizados, lambida = otimizar_precos(indicadora, precos_iniciais)

#if (precos_otimizados < tarifa*0.9).all():
#    precos_otimizados = tarifa
#elif (precos_otimizados > tarifa*1.1).all():
#    precos_otimizados = tarifa
#else: precos_otimizados
        
print("λ:", lambida)
#print("Preços Otimizados:", precos_otimizados)

df2=df
df2["precos_otimizados"]=precos_otimizados
df2['D*']=demanda(df['per_com'],df['per_com_bin_p_1_10'],df['canal_bin_Corretor_Mais'],df['Cod_End_bin_13509'],df['fator_ajuste_bin_f_101_120'],df['classe_bonus_bin_00'],df['precos_otimizados'])

print(df2['Demanda_base'].sum()),
print(df2['D*'].sum()),
print(df2['Demanda_base'].mean()),
print(df2['D*'].mean())
print(df2['premio'].mean()),
print(df2['precos_otimizados'].mean())

λ: 0.0
94.2665999942133
94.2665999942133
0.4713329999710665
0.4713329999710665
2842.338310908
2842.338310908
CPU times: total: 57.9 s
Wall time: 1min 3s


45.63892414018774
45.63892414018774
0.4563892414018774
0.4563892414018774
3155.1168651450002
3155.1168651450002


In [42]:
df2[['Demanda_base','D*','premio','precos_otimizados']]

,Demanda_base,D*,premio,precos_otimizados
5100,0.529919,0.529919,1344.600000,1344.600000
18717,0.443531,0.443531,2683.476898,2683.476898
13215,0.559036,0.559036,2361.825933,2361.825933
4825,0.480005,0.480005,1552.405290,1552.405290
21948,0.355456,0.355456,3727.812509,3727.812509
...,...,...,...,...
798,0.568438,0.568438,825.732588,825.732588
5403,0.740586,0.740586,1678.284455,1678.284455
23737,0.208245,0.208245,5737.454368,5737.454368
6522,0.644348,0.644348,1630.041226,1630.041226


In [143]:
df.to_csv(r'df_total_v2.csv', sep='|' ,header='true')

In [ ]:

# Verificar os preços otimizados e a margem esperada otimizada 
df['precos_otimizados'] = precos_otimizados
df['retencao_otimizada'] = demanda(precos_otimizados,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)
df['retencao_base_esperado']=demanda(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)
df['margem_esperada_otimizada'] =   df['retencao_otimizada']*(df['precos_otimizados'] - df['Act_cost'])

df['margem_esperada_base_case'] = df['retencao_base_esperado']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()
margem_esperada_total_otimizada=df['margem_esperada_otimizada'].sum()

# Exibir os resultados
print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Retenção Média Base Case:", df['retencao_base_esperado'].mean())
print("Premio Medio Otimizado:", df['precos_otimizados'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Tamanho da Base:", df.shape[0])

In [ ]:
#df[['premio','retencao_base_esperado','preco_otimizado','retencao_otimizada']]

In [ ]:
#df#.to_csv(r'C:/Users/vcatelan/Documents/Scripts/dftocsv2.csv', sep='|', encoding='utf-8', header='true')
#df.to_excel(r'df.xlsx', sep='|', encoding='utf-8', header='true')

#criando a função

In [ ]:
df_relatorio_otim = df.copy()

df_relatorio_otim["per_com_pct"] = df_relatorio_otim['per_com']/100
df_relatorio_otim["premio_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["Act_cost_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["Act_cost"]                                                                
df_relatorio_otim["comissao_valor"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["comissao_valor_prob"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["premio_prob"]

apolice_ofertada  = np.round(df_relatorio_otim.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_otim["retencao_otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio_otim["precos_otimizados"])
retencao_estimada = np.mean(df_relatorio_otim["retencao_otimizada"])
premio_esperado   = np.sum(df_relatorio_otim["premio_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
risco_ofertado    = np.mean(df_relatorio_otim["Act_cost"])
risco_esperado    = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
sp_ofertado       = np.sum(df_relatorio_otim["Act_cost"])/np.sum(df_relatorio_otim["precos_otimizados"])
sp_esperado       = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

In [ ]:
df.to_csv(r'df_total.csv', sep='|' ,header='true')

In [ ]:
df_relatorio_otim[['Demanda_base','retencao_otimizada','premio','preco_otimizado']]


In [ ]:
prices_df = pd.DataFrame({
    'chave': df_relatorio["chave"],
    'Act_cost':  df_relatorio["Act_cost"],
    'Premio ofertado com batente':  df_relatorio["Premio_oferecido_com_batente"],
    'Optimized Prices': df_relatorio["premio"],
    'lb_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_relatorio["prob_est"],
    'renewal demand':df_relatorio["demanda otimizada"],
    'variação':df_relatorio["premio"]/df_relatorio["Premio_oferecido_com_batente"]
})
prices_df.head(20)

########

In [34]:
from multiprocessing import Pool
import time
import math

In [48]:
N = df.shape[0]

def func(X, indicadora):
    P = X[:-1]  # Extraindo o vetor de preços [p1, p2, ..., pn]
    L = X[-1]   # Extraindo o multiplicador de Lagrange
    n = len(P)
    
    # Função objetivo e a soma das demandas
    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(indicadora[i,0],
                      indicadora[i,1],
                      indicadora[i,2],
                      indicadora[i,3],
                      indicadora[i,4],
                      indicadora[i,5],
                      P[i])
        objetivo += D_i * (P[i] - custo)
        soma_demandas += D_i
    
    # Definindo a função de Lagrange com a restrição
    return objetivo - L * (soma_demandas - target)

if __name__ == "__main__":
    # first way, using multiprocessing
    start_time = time.perf_counter()
    with Pool() as pool:
      result = pool.map(func,N)
    finish_time = time.perf_counter()
    print("Program finished in {} seconds - using multiprocessing".format(finish_time-start_time))
    print("---")

TypeError: 'int' object is not iterable